<h1>CS152 Assignment 2: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
import numpy as np
import math
from queue import PriorityQueue
# YOUR CODE HERE (OPTIONAL)

#Now, define the class PuzzleNode:
class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle
    """

    def __init__ (self, state,hval,parent = None):
        
        self.initial_state = state.copy()
        #We convert the state of the puzzle into a flat list to make manipulation of its elemtns easier.
        self.state = np.array([state]).flatten()
        self.parent = parent
        self.pruned = False
        self.pos_zero = None
        self.hval = hval

        #gval represents depth level.
        if self.parent is None:
            #If there is no parent than we are depth 0
            self.gval = 0
        else:
            #Otherwise we will just add 1.
            self.gval = self.parent.gval + 1
         
        #Calculating the value of fval by summing up the values of gval and hval
        self.fval = self.gval + self.hval
        
        if self.pos_zero == None:
            """"
            Checks where the psoition of 0 is as that will be our empty space and will help us get open moves
            """
            self.pos_zero = list(np.where(self.state==0))[0]
        
    
    def __lt__(self,other):
        """"
        Method defined for the priority queue.
        """
        return self.fval < other.fval

    def __str__(self):
        """"
        Visualizes the state of the puzzle node.
        """
        return str(self.state)
    

    def possible_moves(self):
        """"
        Will allow us to find the blank space and then using that information provides us with a lsit of moves we
        can make.
        """
        moves = ['L','R','U','D']
        
        x = self.pos_zero[0]
        n = len(self.initial_state)
        
        
        if x % n == 0:
            moves.remove('L')
        if x % n == n-1:
            moves.remove('R')
        if x // n == 0:
            moves.remove('U')
        if x // n == n-1:
            moves.remove('D')

        return moves
    

    def generate_child(self):
        """"
        This method allows us to make successor states (children) which will be based off of the results we get from the
        method possible_moves.
        """
        children=[]
        x = self.pos_zero[0]
        potential_moves = self.possible_moves()
        dimension = int(math.sqrt(len(self.state)))
        
        for action in potential_moves:
            new_state = list(np.array(self.initial_state))
            if action == 'U':
                new_state[x//dimension][x%dimension], new_state[x//dimension - 1][x%dimension] = new_state[x//dimension-1][x%dimension], new_state[x//dimension][x%dimension]
            elif action == 'D':    
                new_state[x//dimension][x%dimension], new_state[x//dimension + 1][x%dimension] = new_state[x//dimension+1][x%dimension], new_state[x//dimension][x%dimension]
            elif action == 'L':
                new_state[x//dimension][x%dimension], new_state[x//dimension][x%dimension-1] = new_state[x//dimension][x%dimension-1], new_state[x//dimension][x%dimension]
            elif action == 'R':
                new_state[x//dimension][x%dimension], new_state[x//dimension][x%dimension+1] = new_state[x//dimension][x%dimension+1], new_state[x//dimension][x%dimension]
            children.append(PuzzleNode(new_state,0,self))
        return children    


    def inv_count(self):
        """"
        Counts the number of ivnersions for the puzzle we have.
        """
        count = 0
        state_list = list(self.state)
        len_st = len(state_list)
        for i in range(len_st):
            for j in range(i+  1,len_st):
                if (state_list[j] and state_list[i] and state_list[i]> state_list[j]):
                    count += 1
        return count
    
    
    #Help from https://www.geeksforgeeks.org/check-instance-15-puzzle-solvable/
    def is_solvable(self):
        """"
        Method used to check if there is asolution to our puzzle or not.
        """
        inv_count = self.inv_count()
        len_ = len(self.initial_state)
        pos_row = int(self.pos_zero)/len_

        if len_%2==1:
            
            return inv_count%2==0
        
        else:
            if pos_row%2==0 and inv_count%2==0:
                
                return True
            elif pos_row %2==1 and inv_count%2==1:
                return True
            else:
                return False

    
    def goal_state(self):
        """"
        Get the goal state for our puzzle.
        """
        n = len(self.state)
        goal = [i for i in range(0,n)]
        return goal
    
    def valid(self):
        """"
        Checker to see if the puzzle we had initally is even valid or not.
        Checks for the range of the values, whether they are uniqye and the dimensions of the original puzzle.
        """
        num = len(self.initial_state)*len(self.initial_state)
        if set(self.state) == set([i for i in range(0,num)]) and self.state.shape[0]== num:
            return True
        return False


<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
# Add any additional code here (e.g. for the memoization extension)
from functools import lru_cache

class memoized():
    def __init__(self, func):
        self.memo = {}
        self.func = func
    def __call__(self, *args):
        if str(args[0]) not in self.memo:
            self.memo[str(args[0])] = self.func(args[0])
        return self.memo[str(args[0])]
    
# Misplaced tiles heuristic
@memoized
def h1(state):

    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    states = np.array(state).flatten()
    n = len(state)
    h = 0;
    
    goal = [i for i in range(0, n*n)]
    
    
    for i, j in zip(states, goal):
        if i != j and i != 0:
            h += 1
             
    return h

# Manhattan distance heuristic
@memoized
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    # YOUR CODE HERE
    h = 0
    states = list(np.array([state]).flatten())
    n = len(state)
    
    for i, val in enumerate(states):
        if val != 0:
            h +=abs(val//n - i//n) + abs(val%n - i%n)
    return h

heuristics = [h1,h2]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
# Import any packages or define any helper functions you need here

def solvePuzzle(state, heuristic):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # YOUR CODE HERE
    
    dimension = len(state)
    
    exp = 0
    max_frontier = 0
    opt_path = []
    cost_database = {}
    err = 0
    steps = 0
    
    
    initial_node = PuzzleNode(state,heuristic(state))
    
    
    if not initial_node.valid():
        #Returns -1 if the state is inconsistent.
        err = -1
    elif not initial_node.is_solvable():
        #Returns -2 if the state is inconsistent.
        err = -2
    
    else:
        frontier = PriorityQueue()
        frontier.put(initial_node)


        while not frontier.empty():
            current_node = frontier.get()

            if current_node.pruned:
                continue

            if list(current_node.state) == current_node.goal_state():
                break
                
            exp += 1
            
            children = current_node.generate_child()

            for child in children:
                gval = child.gval #tentative cost

                if str(child.state) in cost_database:
                    if cost_database[str(child.state)].gval>gval:
                        cost_database[str(child.state)].pruned = True
                    else:
                        continue
                hval = heuristic(child.initial_state)

                next_node = PuzzleNode(child.initial_state,hval,current_node)
                frontier.put(next_node)

                
                cost_database[str(child.state)] = next_node
            max_frontier = max(max_frontier, frontier.qsize())
            
    
    
        previous_parent = current_node.parent
        opt_path = []
        
        opt_path.append([list(i) for i in np.array_split(current_node.state,dimension)])
        while previous_parent:

            opt_path.append([list(i) for i in np.array_split(previous_parent.state,dimension)])
            previous_parent = previous_parent.parent


        steps = len(opt_path)-1
        opt_path = opt_path[::-1]
    return steps,exp,max_frontier,opt_path,err 



<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0])
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1])
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [9]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
    assert(dom > 0)

IndexError: list index out of range

In [ ]:
## Memoization test - will be carried out after submission